In [95]:
import pandas as pd
import requests
import json
import os
from config import geoapify_key
from config import places_key

In [3]:
food_access_data = pd.read_csv("food_access_data.csv")
food_df = pd.DataFrame(food_access_data)

In [4]:
state_list = food_df["State"].unique()
len(food_df)

72531

In [5]:
county_pop_list = []

for state in state_list:
    county_list_test = food_df["County"].loc[food_df["State"] == state].unique()
    for county in county_list_test:
        county_pop = food_df["Pop2010"].loc[(food_df["State"] == state) & (food_df["County"] == county)].sum()
        county_pr = food_df["PovertyRate"].loc[(food_df["State"] == state) & (food_df["County"] == county)].mean()
        county_median_income = food_df["MedianFamilyIncome"].loc[(food_df["State"] == state) & (food_df["County"] == county)].median()
        ind_county_dict = {"State": state,
                           "County": county,
                           "Total Population": county_pop,
                           "Average Poverty Rate": county_pr,
                           "County Median Family Income": county_median_income}
                              
        county_pop_list.append(ind_county_dict)

In [6]:
county_pop_df = pd.DataFrame(county_pop_list)

In [7]:
food_data_final = food_df.merge(county_pop_df, on=["State", "County"])

In [30]:
county_pop_filtered = county_pop_df.loc[county_pop_df["Total Population"] >= 100000]
county_list_sorted = county_pop_filtered.drop(county_pop_filtered[county_pop_filtered["Average Poverty Rate"] == 0.0].index, axis=0).dropna(subset=["Average Poverty Rate", "County Median Family Income"]).sort_values(by = "Average Poverty Rate", ascending = False)

In [31]:
county_list_sorted.head()

,State,County,Total Population,Average Poverty Rate,County Median Family Income
415,Georgia,Clarke County,116714,34.460000,55930.0
2762,Texas,Webb County,250304,33.557377,35767.5
2553,Texas,Cameron County,406220,32.936471,36990.0
2630,Texas,Hidalgo County,774769,31.678571,39875.0
397,Georgia,Bibb County,155547,31.154545,42148.0


In [32]:
county_list_sorted.tail()

,State,County,Total Population,Average Poverty Rate,County Median Family Income
1771,New Hampshire,Rockingham County,295223,4.854688,109606.0
1787,New Jersey,Morris County,492276,4.800000,142232.0
2521,Tennessee,Williamson County,183182,4.243243,128423.0
2872,Virginia,Loudoun County,312311,3.981538,153716.5
262,Colorado,Douglas County,285465,3.718033,133313.0


In [47]:
sample_state_df = pd.concat([county_list_sorted.head(3),county_list_sorted.tail(3)])
sample_state_df

,State,County,Total Population,Average Poverty Rate,County Median Family Income
415,Georgia,Clarke County,116714,34.460000,55930.0
2762,Texas,Webb County,250304,33.557377,35767.5
2553,Texas,Cameron County,406220,32.936471,36990.0
2521,Tennessee,Williamson County,183182,4.243243,128423.0
2872,Virginia,Loudoun County,312311,3.981538,153716.5
262,Colorado,Douglas County,285465,3.718033,133313.0


In [83]:
fast_food_county = []

for i in range(len(sample_state_df)):
    target_type = "locality"
    target_state = str(sample_state_df["State"].iloc[i])
    target_county = str(sample_state_df["County"].iloc[i])
    call_limit = 1
    target_country = "United States"

    geo_params = {"type": target_type,
                  "text": target_county,
                  "state": target_state,
                  "country": target_country,
                  "limit": call_limit,
                  "apiKey": geoapify_key}

    geo_base_url = "https://api.geoapify.com/v1/geocode/search?"
    geo_response = requests.get(geo_base_url, params= geo_params)
    geo_response = geo_response.json()
    
    place_id = geo_response['features'][0]['properties']['place_id']

    target_filter = f"place:{place_id}"
    target_category = "catering.fast_food"
    places_call_limit = 300

    places_params = {"categories": target_category,
                     "filter": target_filter,
                     "limit": places_call_limit,
                     "apiKey": places_key}
    
    places_base_url = "https://api.geoapify.com/v2/places?"
    geo_response = requests.get(places_base_url, params= places_params)
    places_response = geo_response.json()
    fast_food_num = len(places_response["features"])
    fast_food_county.append(fast_food_num)
    print(fast_food_num)


80
96
91
105
192
165


In [84]:
fast_food_county

[80, 96, 91, 105, 192, 165]

In [87]:
sample_state_df["Fast Food Restaurants"] = fast_food_county
sample_state_df["Fast Food per Capita"] = sample_state_df["Fast Food Restaurants"]/sample_state_df["Total Population"]
sample_state_df

,State,County,Total Population,Average Poverty Rate,County Median Family Income,Fast Food Restaurants,Fast Food per Capita
415,Georgia,Clarke County,116714,34.460000,55930.0,80,0.000685
2762,Texas,Webb County,250304,33.557377,35767.5,96,0.000384
2553,Texas,Cameron County,406220,32.936471,36990.0,91,0.000224
2521,Tennessee,Williamson County,183182,4.243243,128423.0,105,0.000573
2872,Virginia,Loudoun County,312311,3.981538,153716.5,192,0.000615
262,Colorado,Douglas County,285465,3.718033,133313.0,165,0.000578


In [94]:
output_file_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'fast_food_data.csv')
sample_state_df.to_csv(output_file_path, index=False)